In [2]:
from pcd_dataset import PointCloudDataset
from plane_detector import StructuralRANSAC
from semantic_segmentation import SemanticSegmentor
from s3dis_metrics import SegmentationMetrics
from pointnet_utils import PointnetInference
import yaml
import numpy as np
import wandb

In [3]:
token = "415dd2b149285b382a2e695e3023f569695b8398"
wandb.login(key=token)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/dmytroivanov/.netrc
wandb: Currently logged in as: ivanovd to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [1]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)

NameError: name 'yaml' is not defined

In [4]:
s_val = PointCloudDataset("../s3DIS/Stanford3dDataset_v1.2_Aligned_Version/")

In [5]:
rooms = ["office_1","office_2","office_3","office_4","office_5","office_6"]

In [6]:
metrics = SegmentationMetrics(config["elements"])
for room in rooms:
    config["room"] = room
    sem_seg = SemanticSegmentor(s_val, StructuralRANSAC, PointnetInference, config)
    colors = np.ones((sem_seg.points.shape[0], 3)) * 0.5
    sem_seg.points[:, 3:] = colors
    sem_seg.ransac_segment()
    sem_seg.refind_walls()
    sem_seg.refined_with_pointnet()
    point_labels = sem_seg.map_ransac_to_s3dis()
    
    results = metrics.compute_metrics(sem_seg.labels, point_labels)
    print(f"Overall Accuracy: {results['overall_acc']:.2%}")

    print("\nPer-class IoU:")
    for class_name, iou in results['iou_per_class'].items():
        print(f"  {class_name}: {iou:.2%}")


Loading ../s3DIS/Stanford3dDataset_v1.2_Aligned_Version//Area_5/office_1 with voxel size 0.02m
  Original points: 816,136
  Subsampled points: 211,052 (25.9%)
Starting structural segmentation...
Detecting floor and ceiling...
  Floor found: 30683 points
  Ceiling found: 40722 points
Segmentation complete in 1.17 seconds
Found: 2 structural elements
  floor: 1
  ceiling: 1
Remaining unsegmented points: 139647
Overall Accuracy: 46.02%

Per-class IoU:
  ceiling: 94.83%
  floor: 92.76%
  wall: 25.13%
  beam: 0.00%
  column: 0.00%
  window: 0.00%
  door: 0.00%
  table: 0.00%
  chair: 0.00%
  sofa: 0.00%
  bookcase: 0.00%
  board: 0.00%
  clutter: 11.26%
Loading ../s3DIS/Stanford3dDataset_v1.2_Aligned_Version//Area_5/office_2 with voxel size 0.02m
  Original points: 867,248
  Subsampled points: 222,541 (25.7%)
Starting structural segmentation...
Detecting floor and ceiling...
  Floor found: 30351 points
  Ceiling found: 38205 points
Segmentation complete in 1.29 seconds
Found: 2 structural e

In [7]:
metrics.compute_mean_iou()

In [8]:
metrics.results

{'ceiling': np.float64(0.9476119425043698),
 'floor': np.float64(0.933290254045072),
 'wall': np.float64(0.3513755939955447),
 'window': np.float64(0.0),
 'door': np.float64(0.0)}

In [9]:
metrics.mean_iou

np.float64(0.44645555810899734)

In [10]:
config["room"] = rooms
run = wandb.init(
    
    project="point-cloud-sem-seg",
    
    config=config,
)
run.log({"mean_iou": metrics.mean_iou})
run.log(metrics.results)
run.finish()

ceiling,▁
door,▁
floor,▁
mean_iou,▁
wall,▁
window,▁
ceiling,0.94761
door,0
floor,0.93329
mean_iou,0.44646
wall,0.35138


In [11]:
metrics = SegmentationMetrics(config["elements"])
for room in rooms:
    config["room"] = room
    sem_seg = SemanticSegmentor(s_val, StructuralRANSAC, PointnetInference, config)
    colors = np.ones((sem_seg.points.shape[0], 3)) * 0.5
    sem_seg.points[:, 3:] = colors
    predictions = sem_seg.pointnet_segmentation(sem_seg.points, "all")
    results = metrics.compute_metrics(sem_seg.labels, predictions)
    print(f"Overall Accuracy: {results['overall_acc']:.2%}")

    print("\nPer-class IoU:")
    for class_name, iou in results['iou_per_class'].items():
        print(f"  {class_name}: {iou:.2%}")


Loading ../s3DIS/Stanford3dDataset_v1.2_Aligned_Version//Area_5/office_1 with voxel size 0.02m
  Original points: 816,136
  Subsampled points: 211,052 (25.9%)
Overall Accuracy: 42.52%

Per-class IoU:
  ceiling: 74.79%
  floor: 76.88%
  wall: 24.69%
  beam: 0.00%
  column: 0.00%
  window: 0.00%
  door: 0.00%
  table: 0.45%
  chair: 0.00%
  sofa: 0.00%
  bookcase: 0.40%
  board: 0.00%
  clutter: 10.03%
Loading ../s3DIS/Stanford3dDataset_v1.2_Aligned_Version//Area_5/office_2 with voxel size 0.02m
  Original points: 867,248
  Subsampled points: 222,541 (25.7%)
Overall Accuracy: 45.15%

Per-class IoU:
  ceiling: 66.58%
  floor: 66.08%
  wall: 40.56%
  beam: 0.00%
  column: 0.00%
  window: 0.00%
  door: 0.00%
  table: 0.14%
  chair: 0.00%
  sofa: 0.00%
  bookcase: 13.99%
  board: 0.00%
  clutter: 7.75%
Loading ../s3DIS/Stanford3dDataset_v1.2_Aligned_Version//Area_5/office_3 with voxel size 0.02m
  Original points: 760,563
  Subsampled points: 196,973 (25.9%)
Overall Accuracy: 42.24%

Per-cla

In [12]:
metrics.compute_mean_iou()

In [13]:
metrics.results

{'ceiling': np.float64(0.7256079837020518),
 'floor': np.float64(0.6969765591735532),
 'wall': np.float64(0.3623456390486371),
 'window': np.float64(0.0),
 'door': np.float64(0.0)}

In [14]:
metrics.mean_iou

np.float64(0.3569860363848484)

In [15]:
config["approach"] = "pointnet"

In [16]:
config["room"] = rooms
run = wandb.init(
    
    project="point-cloud-sem-seg",
    
    config=config,
)
run.log({"mean_iou": metrics.mean_iou})
run.log(metrics.results)
run.finish()

ceiling,▁
door,▁
floor,▁
mean_iou,▁
wall,▁
window,▁
ceiling,0.72561
door,0
floor,0.69698
mean_iou,0.35699
wall,0.36235
